In [1]:
import pandas as pd
print(9)
df = pd.read_csv("Data/wineSet.csv")
df.head()

9


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [2]:
df.info()
#find class imbalanc
df.taster_name.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


Roger Voss            25514
Michael Schachner     15134
Kerin O’Keefe         10776
Virginie Boone         9537
Paul Gregutt           9532
Matt Kettmann          6332
Joe Czerwinski         5147
Sean P. Sullivan       4966
Anna Lee C. Iijima     4415
Jim Gordon             4177
Anne Krebiehl MW       3685
Lauren Buzzeo          1835
Susan Kostrzewa        1085
Mike DeSimone           514
Jeff Jenssen            491
Alexander Peartree      415
Carrie Dykes            139
Fiona Adams              27
Christina Pickard         6
Name: taster_name, dtype: int64

In [4]:
import numpy as np
#topicsByReview = pd.DataFrame(rows, columns=["review","points","description","topic"])

df = df.drop(df[df.taster_name == "Virginie Boone" or df.taster_name == "Paul Gregutt"].index)


df.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
from sklearn.model_selection import train_test_split

X = df["description"]
y = df["label"]
#variablen werden direkt aus der Funktion, random_state damit immer gleiche aufteilung gemacht wird
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=17)

#Vektor aus Text erstellen: Diskriminierung, Wörter die zu häufig und zu selten vorkommen sind schlecht zur Diskriminierung weil damit nicht erkennt
#werden kann ob es sich um die einte Klasse Text oder um die andere Klasse handelt
# Stoppwort Elimination wird dazu verwendet damit nurnoch relevante Wörter(Index Terms) übrig sind
#Vektor mit den wichtigen Wörter Bilden 
#Es braucht eine Voranalyse in welcher alle Stoppwörter eliminiert und die Index Terms bestimmt

#Aus Text einen Vektor machen
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#"tfidf" und "model" sind keine KeyWords und können einfach irgendwie beannt werden, effektiv wird von tfidf ein 
# Vektor mit beliebig vielen Feature an das model übergeben
pipe = Pipeline([("tfidf",TfidfVectorizer()), ("model",MultinomialNB())])

pipe.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('model', MultinomialNB())])

In [12]:
#Test model
from sklearn import metrics

predictions = pipe.predict(X_test)
accuracy = metrics.accuracy_score(y_true=y_test, y_pred=predictions)
print(accuracy)
confusion = metrics.confusion_matrix(y_true=y_test, y_pred=predictions)
print(confusion)

0.8002308136180034
[[11348  2485]
 [ 2708  9454]]


In [13]:
q = "nothing look forward lifei dont many reasons"
a = pipe.predict([q])
print(a)

['negative']
